In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(conn, reflect = True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Mea = Base.classes.measurement
Sta = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(conn)

# Exploratory Precipitation Analysis

In [ ]:

# Find the most recent date in the data set.
recent_date = session.query(Mea).order_by(Mea.date.desc()).first().__dict__
recent_date
inspector = inspect(conn)
inspector.get_table_names()
column_info = inspector.get_columns('measurement')
column_info
for column in column_info:
    print(column['name'], column['type'],column['primary_key'])

In [ ]:
date, = session.query(Mea.date).order_by(Mea.date.desc()).first()
date

In [ ]:
for columns in inspector.get_columns('station'):
    print(columns['name'],
          columns['type'],
          columns['primary_key']
          )


In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.

#Parse the recent date with datetime
recent_parsed = dt.datetime.strptime(recent_date['date'],"%Y-%m-%d")
#Create a new string while reducing year by one and parsing the month with 0 before the number.
target_date = f"{recent_parsed.year-1}-{recent_parsed.month:02}-{recent_parsed.day:02}"

# Run the session
result = session.query(Mea).filter(
    func.strftime("%Y-%m-%d",Mea.date) > target_date
    ).all()

# Perform a query to retrieve the data and precipitation scores
precipation_list = []
for row in result:
    precipation_list.append([row.date,row.prcp])
precipation_list
# Save the query results as a Pandas DataFrame and set the index to the date column
preci_df = pd.DataFrame(precipation_list, columns= ['Date','Precipitation'])

# Sort the dataframe by date
prec_df_sort = preci_df.set_index('Date')

# Use Pandas Plotting with Matplotlib to plot the data
prec_df_sort.plot(figsize = (7,4))
plt.title('Percipitation Between 2016 and 2017')
plt.ylabel('Percipitation')
plt.xticks(rotation = 45)
plt.savefig("Images/precipitation.png")

plt.show()


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
prec_df_sort.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_tot = session.query(Sta).count()
station_tot


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
active_stations = session.query(Mea.station, func.count(Mea.id)).group_by(Mea.station).order_by(func.count(Mea.id).desc()).all()
active_stations
# for stations in active_stations:
#     active_station_list.append(stations[0].__dict__['station'])
# active_station_list

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
station_temp = session.query(Mea.station,func.min(Mea.tobs),func.max(Mea.tobs),func.avg(Mea.tobs)).group_by(Mea.station).order_by(func.count(Mea.id).desc()).all()
station_temp

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
active_station_data = session.query(Mea.date,Mea.tobs).filter(Mea.station == station_temp[0][0]).all()
station_df = pd.DataFrame(active_station_data, columns = ['Date','tobs'])
station_sort = station_df.set_index('Date')
station_sort.plot.hist(figsize = (7,4), bins=12)
plt.title(f'Common Temperature for Station {station_temp[0][0]}')
plt.xlabel('Tempearture')
plt.savefig('Images/active_station_info.png')

plt.show()

# Close session

In [ ]:
# Close Session
session.close()